In [1]:
random_state = 16

## Imports

In [2]:
!pip install scikit-learn xgboost 

In [3]:
!pip install pandas numpy seaborn matplotlib yellowbrick scipy

In [4]:
#imports
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt    
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import RFE, RFECV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from yellowbrick.regressor import PredictionError, ResidualsPlot
from yellowbrick.style import set_palette
from yellowbrick.style import rcmod
from sklearn.feature_selection import SelectFromModel

## Data Preprocessing

In [5]:
!ls

clusters2.csv	       esm_p2_xgb.txt		mlp_1.png
DTC_esm_P1.png	       esm_pipeline_1.ipynb	mlp_2.png
DTC_esm_P2.png	       esm_pipeline_2.ipynb	mlp_esm_1.png
EN_esm_P2.png	       esm_preprocessing.ipynb	mlp_esm_2.png
en_esm.png	       esm_protherm_1280.csv	preprocessing.ipynb
esm_complete_1280.csv  esm_test_1280.csv	protherm_deep_physchem.csv
esm_complete.csv       esm_test.csv		protherm.xlsx
esm_features.csv       esm_train_1280.csv	zero_features_1280.csv
esm_groups_1280.csv    esm_train.csv		zero_features_esm.csv
esm_groups.csv	       KNN_esm_P1.png
esm_p1_xgb.txt	       KNN_esm_P2.png


In [7]:
data = pd.read_csv("esm_complete_1280.csv")
data.head()

,Protein_ID,Tm_C,0,1,2,3,4,5,6,7,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,P00350,57.83,0.061456,-0.018783,-0.010231,0.061477,-0.026708,-0.001284,0.066149,-0.251243,...,0.003973,-0.101666,-0.036435,-0.049548,-0.001934,0.003177,0.043629,-0.133913,0.007773,0.049505
1,P00363,46.77,0.034969,0.042067,0.013021,-0.032451,-0.087499,-0.042588,0.037960,-0.183190,...,-0.032316,-0.082484,-0.027231,-0.032674,-0.032110,0.050828,0.059285,-0.132150,0.012745,0.077547
2,P00370,58.78,0.053757,-0.021765,-0.029727,0.024891,-0.024003,-0.017309,0.041215,-0.213883,...,-0.009158,-0.060221,-0.066123,-0.024452,0.011502,0.016854,0.007332,-0.101124,-0.010835,0.105137
3,P00448,66.59,-0.026329,-0.123985,-0.002493,0.070453,-0.023620,-0.072184,0.017002,-0.184937,...,0.020440,-0.048012,-0.023534,0.022797,-0.054544,-0.051367,0.041085,-0.204482,-0.001493,0.067742
4,P00452,44.79,-0.030523,-0.028832,0.017711,0.119092,-0.099517,0.040022,0.028322,-0.127345,...,0.140361,-0.007807,-0.133724,0.026237,-0.004347,0.040577,-0.003302,-0.161196,-0.032264,0.173568


In [8]:
data.shape

(6672, 1282)

In [9]:
data = data[data['Tm_C'] != 0]
data.shape

(6672, 1282)

In [10]:
data = data.reset_index(drop = True)

## Clustering

In [11]:
clusters = pd.read_csv('clusters2.csv', index_col = 0)
clusters.head()

,cluster#,Protein_ID
0,0,P00363
1,1,P00370
2,2,P00509
3,3,P00582
4,4,P00634


In [12]:
clusters.shape, data.shape

((6672, 2), (6672, 1282))

In [13]:
data_set = pd.merge(data, clusters, on = 'Protein_ID')
data_set.shape

(6672, 1283)

In [14]:
data_set.head()

,Protein_ID,Tm_C,0,1,2,3,4,5,6,7,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,cluster#
0,P00350,57.83,0.061456,-0.018783,-0.010231,0.061477,-0.026708,-0.001284,0.066149,-0.251243,...,-0.101666,-0.036435,-0.049548,-0.001934,0.003177,0.043629,-0.133913,0.007773,0.049505,3092
1,P00363,46.77,0.034969,0.042067,0.013021,-0.032451,-0.087499,-0.042588,0.037960,-0.183190,...,-0.082484,-0.027231,-0.032674,-0.032110,0.050828,0.059285,-0.132150,0.012745,0.077547,0
2,P00370,58.78,0.053757,-0.021765,-0.029727,0.024891,-0.024003,-0.017309,0.041215,-0.213883,...,-0.060221,-0.066123,-0.024452,0.011502,0.016854,0.007332,-0.101124,-0.010835,0.105137,1
3,P00448,66.59,-0.026329,-0.123985,-0.002493,0.070453,-0.023620,-0.072184,0.017002,-0.184937,...,-0.048012,-0.023534,0.022797,-0.054544,-0.051367,0.041085,-0.204482,-0.001493,0.067742,734
4,P00452,44.79,-0.030523,-0.028832,0.017711,0.119092,-0.099517,0.040022,0.028322,-0.127345,...,-0.007807,-0.133724,0.026237,-0.004347,0.040577,-0.003302,-0.161196,-0.032264,0.173568,3093


In [15]:
X = data_set.drop(columns=['Tm_C', 'cluster#'], axis=1).copy()
y = data_set['Tm_C'].copy()
groups = data_set['cluster#']

In [16]:
groups = data_set['cluster#']
print('Total number of different clusters: {}'.format(max(groups)))

Total number of different clusters: 5778


In [17]:
gkf = GroupKFold(n_splits=5)
for train_index, test_index in gkf.split(X, y, groups):
    print(train_index, test_index)
    

[   0    1    2 ... 6669 6670 6671] [   8   14   17 ... 6654 6661 6667]
[   0    1    2 ... 6669 6670 6671] [   6    9   23 ... 6656 6662 6668]
[   2    4    5 ... 6669 6670 6671] [   0    1    3 ... 6655 6657 6663]
[   0    1    3 ... 6668 6669 6670] [   2    5    7 ... 6660 6664 6671]
[   0    1    2 ... 6667 6668 6671] [   4   16   18 ... 6666 6669 6670]


In [18]:
data_set_train = data_set.iloc[train_index]
data_set_test = data_set.drop(columns='cluster#').iloc[test_index]

In [19]:
groups = data_set_train['cluster#']

In [20]:
data_set_train.shape, data_set_test.shape, groups.shape

((5338, 1283), (1334, 1282), (5338,))

In [21]:
data_set_train = data_set_train.drop(columns='cluster#')
data_set_train.to_csv("esm_train_1280.csv")

In [22]:
data_set_test.to_csv("esm_test_1280.csv")

In [23]:
groups.to_csv("esm_groups_1280.csv")